In [31]:
# look at tools/set_up_magics.ipynb
yandex_metrica_allowed = True ; get_ipython().run_cell('# one_liner_str\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport re\nimport signal\nimport shutil\nimport shlex\nimport glob\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                if line.startswith(run_prefix):\n                    cmds.append(line[len(run_prefix):].strip())\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                run_prefix = "%# "\n                if line.startswith(run_prefix):\n                    f.write(line_comment_start + " " + line_to_write)\n                    continue\n                raise Exception("Unknown %%save_file subcommand: \'%s\'" % line)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \n    \ndef show_log_file(file, return_html_string=False):\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function halt__OBJ__(elem, color)\n        {\n            elem.setAttribute("style", "font-size: 14px; background: " + color + "; padding: 10px; border: 3px; border-radius: 5px; color: white; ");                    \n        }\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            if (entrance___OBJ__ < 0) {\n                entrance___OBJ__ = 0;\n            }\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                if (elem.innerHTML != xmlhttp.responseText) {\n                                    elem.innerHTML = xmlhttp.responseText;\n                                }\n                                if (elem.innerHTML.includes("Process finished.")) {\n                                    halt__OBJ__(elem, "#333333");\n                                } else {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                }\n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (!entrance___OBJ__) {\n                                if (errors___OBJ__ < 6) {\n                                    entrance___OBJ__ += 1;\n                                    console.log("req");\n                                    window.setTimeout("refresh__OBJ__()", 300); \n                                } else {\n                                    halt__OBJ__(elem, "#994444");\n                                }\n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n\n        <p id="__OBJ__" style="font-size: 14px; background: #000000; padding: 10px; border: 3px; border-radius: 5px; color: white; ">\n        </p>\n        \n        </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__.md -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n\n    \nclass TInteractiveLauncher:\n    tmp_path = "./interactive_launcher_tmp"\n    def __init__(self, cmd):\n        try:\n            os.mkdir(TInteractiveLauncher.tmp_path)\n        except:\n            pass\n        name = str(random.randint(0, 1e18))\n        self.inq_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".inq")\n        self.log_path = os.path.join(TInteractiveLauncher.tmp_path, name + ".log")\n        \n        os.mkfifo(self.inq_path)\n        open(self.log_path, \'w\').close()\n        open(self.log_path + ".md", \'w\').close()\n\n        self.pid = os.fork()\n        if self.pid == -1:\n            print("Error")\n        if self.pid == 0:\n            exe_cands = glob.glob("../tools/launcher.py") + glob.glob("../../tools/launcher.py")\n            assert(len(exe_cands) == 1)\n            assert(os.execvp("python3", ["python3", exe_cands[0], "-l", self.log_path, "-i", self.inq_path, "-c", cmd]) == 0)\n        self.inq_f = open(self.inq_path, "w")\n        interactive_launcher_opened_set.add(self.pid)\n        show_log_file(self.log_path)\n\n    def write(self, s):\n        s = s.encode()\n        assert len(s) == os.write(self.inq_f.fileno(), s)\n        \n    def get_pid(self):\n        n = 100\n        for i in range(n):\n            try:\n                return int(re.findall(r"PID = (\\d+)", open(self.log_path).readline())[0])\n            except:\n                if i + 1 == n:\n                    raise\n                time.sleep(0.1)\n        \n    def input_queue_path(self):\n        return self.inq_path\n        \n    def close(self):\n        self.inq_f.close()\n        os.waitpid(self.pid, 0)\n        os.remove(self.inq_path)\n        # os.remove(self.log_path)\n        self.inq_path = None\n        self.log_path = None \n        interactive_launcher_opened_set.remove(self.pid)\n        self.pid = None\n        \n    @staticmethod\n    def terminate_all():\n        if "interactive_launcher_opened_set" not in globals():\n            globals()["interactive_launcher_opened_set"] = set()\n        global interactive_launcher_opened_set\n        for pid in interactive_launcher_opened_set:\n            print("Terminate pid=" + str(pid), file=sys.stderr)\n            os.kill(pid, signal.SIGKILL)\n            os.waitpid(pid, 0)\n        interactive_launcher_opened_set = set()\n        if os.path.exists(TInteractiveLauncher.tmp_path):\n            shutil.rmtree(TInteractiveLauncher.tmp_path)\n    \nTInteractiveLauncher.terminate_all()\n   \nyandex_metrica_allowed = bool(globals().get("yandex_metrica_allowed", False))\nif yandex_metrica_allowed:\n    display(HTML(\'\'\'<!-- YANDEX_METRICA_BEGIN -->\n    <script type="text/javascript" >\n       (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};\n       m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})\n       (window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");\n\n       ym(59260609, "init", {\n            clickmap:true,\n            trackLinks:true,\n            accurateTrackBounce:true\n       });\n    </script>\n    <noscript><div><img src="https://mc.yandex.ru/watch/59260609" style="position:absolute; left:-9999px;" alt="" /></div></noscript>\n    <!-- YANDEX_METRICA_END -->\'\'\'))\n\ndef make_oneliner():\n    html_text = \'("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "")\'\n    html_text += \' + "<""!-- MAGICS_SETUP_PRINTING_END -->"\'\n    return \'\'.join([\n        \'# look at tools/set_up_magics.ipynb\\n\',\n        \'yandex_metrica_allowed = True ; get_ipython().run_cell(%s);\' % repr(one_liner_str),\n        \'display(HTML(%s))\' % html_text,\n        \' #\'\'MAGICS_SETUP_END\'\n    ])\n       \n\n');display(HTML(("В этот ноутбук встроен код Яндекс Метрики для сбора статистики использований. Если вы не хотите, чтобы по вам собиралась статистика, исправьте: yandex_metrica_allowed = False" if yandex_metrica_allowed else "") + "<""!-- MAGICS_SETUP_PRINTING_END -->")) #MAGICS_SETUP_END

<IPython.core.display.Javascript object>

# goto на максималках и как при segfault отряхнуться и работать дальше



<p><a href="https://17" target="_blank">
    <h3>Видеозапись семинара</h3> 
</a></p>

Сегодня будем рассматривать:
* <a href="#goto" style="color:#856024">`goto`</a>
* <a href="#jmp" style="color:#856024">`setjmp` и `longjmp`</a>
* <a href="#segfault" style="color:#856024">Как обработать segfault</a>

# goto

Еще раз напомню - хорошо думайте перед использованием goto, нужен ли он вам тут или можно обойтись без него. Так как его использование сильно повышает вероятность ошибок и неправильного понимания кода читающими.

In [32]:
%%cpp go.cpp
%run gcc go.cpp -o go.exe
%run ./go.exe

#include <stdio.h>

int main() {
    int i = 0;
    start:
    printf("i = %d\n", i);
    if (++i < 3) {
        goto start;
    }
    return 0;
}

Run: `gcc go.cpp -o go.exe`

Run: `./go.exe`

i = 0
i = 1
i = 2


# <a name="jmp"></a> `setjmp` и `longjmp`

In [33]:
%%cpp jmp.cpp
%run gcc jmp.cpp -o jmp.exe
%run ./jmp.exe

#include <stdio.h>
#include <setjmp.h>


int main() {
    int i = 0;
    jmp_buf environment;
    int jmp_ret = setjmp(environment);
    printf("i = %d, jmp_ret=%d\n", i, jmp_ret);

    if (++i < 3) {
        longjmp(environment, i * 100 + 1);
    }
    return 0;
}

Run: `gcc jmp.cpp -o jmp.exe`

Run: `./jmp.exe`

i = 0, jmp_ret=0
i = 1, jmp_ret=101
i = 2, jmp_ret=201


In [34]:
%%cpp jmp2.cpp
%run gcc jmp2.cpp -o jmp2.exe
%run ./jmp2.exe

#include <stdio.h>
#include <setjmp.h>

jmp_buf environment;

void f(int i) {
    if (i == 3) {
        longjmp(environment, 1);
    }
    printf("i = %d\n", i);
    f(i + 1);
    printf("After recursive call 1\n");
}

int main() {
    int i = 0;
    
    if (setjmp(environment) == 0) {
        printf("Right after setjmp\n");
        f(0);
        printf("After recursive call 1\n");
    } else {
        printf("Right after return to setjmp\n");
    }
    return 0;
}

Run: `gcc jmp2.cpp -o jmp2.exe`

Run: `./jmp2.exe`

Right after setjmp
i = 0
i = 1
i = 2
Right after return to setjmp


# <a name="segfault"></a> Как обработать segfault

SIGSEGV - интересный сигнал, при получении сегфолта он обрабатывается немедленно, что открывает интересные возможности.

При подготовке примера опирался на вот это: https://habr.com/ru/post/332626/

In [38]:
%%cpp segv.cpp
%run gcc segv.cpp -o segv.exe
%run ./segv.exe

#include <stdio.h>
#include <setjmp.h>
#include <assert.h>
#include <inttypes.h>
#include <signal.h>


jmp_buf segv_catcher_jmp_buf;

void segv_catcher_handler(int signum)
{
    // разблокируем пришедший сигнал вручную, 
    // так как из обработчика мы не выйдем нормальным образом
    sigset_t signals;
    sigemptyset(&signals);
    sigaddset(&signals, signum);
    sigprocmask(SIG_UNBLOCK, &signals, nullptr);
    
    dprintf(2, "Got signal %d\n", signum);
    
    longjmp(segv_catcher_jmp_buf, 1);
}


void segv_catcher_init() {
    // отдельный стек для обработчика сигнала
    // иначе при получении сегфолта из-за переполнения стека 
    // обработчику сигнала будет негде работать
    static uint8_t segv_catcher_stack[65536];
    stack_t ss;
    ss.ss_sp = segv_catcher_stack;
    ss.ss_flags = 0;
    ss.ss_size = sizeof(segv_catcher_stack);
    sigaltstack(&ss, NULL);
    
    // обработчик сигнала сегфолта
    struct sigaction sa{};
    sigemptyset(&sa.sa_mask);
    sa.sa_flags = SA_ONSTACK;
    sa.sa_handler = segv_catcher_handler;
    sigaction(SIGSEGV, &sa, NULL);
}

int f(int i) {
    return f(i + 1) + 1;
}

int f2(int i) {
    return i > 10 ? 1 : f2(i + 1) + 1;
}

int main() {
    segv_catcher_init();
    
    if (!setjmp(segv_catcher_jmp_buf)) {
        printf("Call f()\n");
        int res = f(0);
        printf("Success call of f(0) = %d\n", res);
    } else {
        printf("Failed call of f() - segfault\n");
    }
    
    if (!setjmp(segv_catcher_jmp_buf)) {
        printf("Call f()\n");
        int res = f(0);
        printf("Success call of f(0) = %d\n", res);
    } else {
        printf("Failed call of f() - segfault\n");
    }
    
    if (!setjmp(segv_catcher_jmp_buf)) {
        printf("Call f2()\n");
        int res = f2(0);
        printf("Success call of f2(0) = %d\n", res);
    } else {
        printf("Failed call of f2() - segfault\n");
    }
    
    return 0;
}

Run: `gcc segv.cpp -o segv.exe`

Run: `./segv.exe`

Call f()
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 

Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got si

Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got si

...

Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got si

Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got si

Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got si

Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got si

ignal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 11
Got signal 

Таким же образом можно перехватывать SIGBUS, SIGFPE, SIGILL

In [ ]:
%%cpp ex0

volatile sig_atomic_t sigIntAmount = 0;
volatile sig_atomic_t sigTermAmount = 0;
void controller(int signal)
{
    if (signal == SIGTERM) {
        sigTermAmount++;
    }
    ...
}

int main()
{
    struct sigaction action;
    sigaction(SIGINT, ...controller...);
    sigaction(SIGTERM, ...controller...);
    printf("%d\n", getpid());
    fflush(stdout);
    while (sigTermAmount == 0) { pause();}
    printf("%d\n", sigIntAmount);
    return 0;
}


In [ ]:
%%cpp ex1

sigaction(SIGALRM, ...trivial_handler...);
alarm(time_in_seconds);
int status;
pid_t daughter_process_pid = waitpid(pid, &status, 0);

В следующем примере все корректно будет работать. 
Но написано что-то сильно странное, найдите что.

In [40]:
%%cpp ex2

mkfifo(fifo_name, 0666);
kill(proc_to_inform, SIGHUP);

int fd = -1;
while (1) {
    fd = open(fifo_name, O_WRONLY);
    if (fd >= 0) {
        break;
    }
}

Отрывок из обработчика сигнала. Найдите странность

In [39]:
%%cpp ex3
int size = snprintf (buffer, sizeof (buffer), "%d\n", counter);
write (1, buffer, size);
fflush (stdout);

In [ ]:
volatile sig_atomic_t counter = 0;   // counter of incoming sigints

void handle_exit(int signum) {

    fflush(stdout);
    exit(0);
}

void handle_sigusr(int signum) {
    if (signum == SIGUSR1) {
        counter++;
    } else {
        counter*= -1;
    }
    printf("%d\n", counter);
    fflush(stdout);
}


int main() {
    sigaction(SIGINT, ...);
    sigaction(SIGTERM, ...);
    sigaction(SIGUSR1, ...);
    sigaction(SIGUSR2, ...);

    printf("%d\n", getpid());
    fflush(stdout);

    scanf("%d", &counter);

    while (1) {
        pause();
    }
    return 0;
}

In [41]:
%%cpp ex5

void controller(int signal)
{
    exit();
}

int main()
{
    sigaction(SIGINT, ...controller...);

    scanf("%d", &answer);
    printf("%d\n", getpid());
    fflush(stdout);
    
    while (signalAmount == 0) {
        pause();
    }
    return 0;
}